In [ ]:
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder.getOrCreate()
spark

In [14]:
df = spark.read.format('csv').option('header', 'true').load('data/cars.csv')

In [18]:
df.show(1, vertical=True)

-RECORD 0-------------------------
 manufacturer_name | Subaru       
 model_name        | Outback      
 transmission      | automatic    
 color             | silver       
 odometer_value    | 190000       
 year_produced     | 2010         
 engine_fuel       | gasoline     
 engine_has_gas    | False        
 engine_type       | gasoline     
 engine_capacity   | 2.5          
 body_type         | universal    
 has_warranty      | False        
 state             | owned        
 drivetrain        | all          
 price_usd         | 10900.0      
 is_exchangeable   | False        
 location_region   | Минская обл. 
 number_of_photos  | 9            
 up_counter        | 13           
 feature_0         | False        
 feature_1         | True         
 feature_2         | True         
 feature_3         | True         
 feature_4         | False        
 feature_5         | True         
 feature_6         | False        
 feature_7         | True         
 feature_8         |

Select statement

In [22]:
df.select("feature_0", "duration_listed").show(5)

+---------+---------------+
|feature_0|duration_listed|
+---------+---------------+
|    False|             16|
|    False|             83|
|    False|            151|
|     True|             86|
|    False|              7|
+---------+---------------+
only showing top 5 rows



In [24]:
df.select(df["feature_0"], df.duration_listed).show(5) # as pandas

+---------+---------------+
|feature_0|duration_listed|
+---------+---------------+
|    False|             16|
|    False|             83|
|    False|            151|
|     True|             86|
|    False|              7|
+---------+---------------+
only showing top 5 rows



In [26]:
import pyspark.sql.functions as F # another way to select with spark functions

In [28]:
df.select(F.col("feature_0"), F.col("duration_listed")).show(5) # we have a method, so we can keep working 

+---------+---------------+
|feature_0|duration_listed|
+---------+---------------+
|    False|             16|
|    False|             83|
|    False|            151|
|     True|             86|
|    False|              7|
+---------+---------------+
only showing top 5 rows



Filter statement

In [35]:
df.filter("manufacturer_name = 'Audi'").show(1)

+-----------------+----------+------------+-----+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------------+
|manufacturer_name|model_name|transmission|color|odometer_value|year_produced|engine_fuel|engine_has_gas|engine_type|engine_capacity|body_type|has_warranty|state|drivetrain|price_usd|is_exchangeable|location_region|number_of_photos|up_counter|feature_0|feature_1|feature_2|feature_3|feature_4|feature_5|feature_6|feature_7|feature_8|feature_9|duration_listed|
+-----------------+----------+------------+-----+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+-------

In [41]:
df.select("manufacturer_name", "feature_0", "duration_listed")\
.filter("(manufacturer_name = 'Audi' and transmission = 'mechanical') or manufacturer_name = 'Subaru'").show(5)

+-----------------+---------+---------------+
|manufacturer_name|feature_0|duration_listed|
+-----------------+---------+---------------+
|           Subaru|    False|             16|
|           Subaru|    False|             83|
|           Subaru|    False|            151|
|           Subaru|     True|             86|
|           Subaru|    False|              7|
+-----------------+---------+---------------+
only showing top 5 rows



In [44]:
df.select("manufacturer_name", "feature_0", "duration_listed")\
.filter(F.col("transmission") == 'mechanical')\
.filter(F.col('manufacturer_name') == 'Subaru').show(5)

+-----------------+---------+---------------+
|manufacturer_name|feature_0|duration_listed|
+-----------------+---------+---------------+
|           Subaru|     True|             86|
|           Subaru|    False|             11|
|           Subaru|    False|              2|
|           Subaru|     True|            230|
|           Subaru|    False|             21|
+-----------------+---------+---------------+
only showing top 5 rows



Count

In [46]:
df.distinct().count()

38491

In [48]:
df.select(F.col('manufacturer_name')).distinct().count()

55

groupBy orderBy

In [52]:
df.groupBy('manufacturer_name').count().orderBy(F.col('count').desc()).show() # F.col('count').desc() without function it's impossible

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|             Opel| 2759|
|              BMW| 2610|
|             Ford| 2566|
|          Renault| 2493|
|             Audi| 2468|
|    Mercedes-Benz| 2237|
|          Peugeot| 1909|
|          Citroen| 1562|
|           Nissan| 1361|
|            Mazda| 1328|
|           Toyota| 1246|
|          Hyundai| 1116|
|            Skoda| 1089|
|              Kia|  912|
|       Mitsubishi|  887|
|             Fiat|  824|
|            Honda|  797|
|            Volvo|  721|
|              ВАЗ|  481|
+-----------------+-----+
only showing top 20 rows



In [53]:
df.withColumnRenamed().select('manufacturer').show(5)

+------------+
|manufacturer|
+------------+
|      Subaru|
|      Subaru|
|      Subaru|
|      Subaru|
|      Subaru|
+------------+
only showing top 5 rows



In [54]:
df.withColumn("next_year", F.col("year_produced") + 1).select("year_produced", "next_year").show(5)

+-------------+---------+
|year_produced|next_year|
+-------------+---------+
|         2010|   2011.0|
|         2002|   2003.0|
|         2001|   2002.0|
|         1999|   2000.0|
|         2001|   2002.0|
+-------------+---------+
only showing top 5 rows



printSchema describe

In [55]:
df.printSchema()

root
 |-- manufacturer_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- color: string (nullable = true)
 |-- odometer_value: string (nullable = true)
 |-- year_produced: string (nullable = true)
 |-- engine_fuel: string (nullable = true)
 |-- engine_has_gas: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- engine_capacity: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- has_warranty: string (nullable = true)
 |-- state: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- price_usd: string (nullable = true)
 |-- is_exchangeable: string (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: string (nullable = true)
 |-- up_counter: string (nullable = true)
 |-- feature_0: string (nullable = true)
 |-- feature_1: string (nullable = true)
 |-- feature_2: string (nullable = true)
 |-- feature_3: string (nullable = true)


In [59]:
df.select('manufacturer_name', 'year_produced').describe().show() # fast metrics

+-------+-----------------+------------------+
|summary|manufacturer_name|     year_produced|
+-------+-----------------+------------------+
|  count|            38531|             38531|
|   mean|             null|2002.9437336170874|
| stddev|             null| 8.065730511309935|
|    min|            Acura|              1942|
|    max|              УАЗ|              2019|
+-------+-----------------+------------------+

